# Evidenz zur Klassifikation von Wohnungsbewohnerkategorien (Datensatz "wohnungen_1.csv")
Dieses Notebook stellt ein Modell auf Basis der Evidenztheorie vor, das verwendet wird, um die Bewohnerkategorie einer Wohnung basierend auf den Merkmalen der Wohnung zu klassifizieren.

## Ziel
Das Ziel dieses Projekts ist es, die Zusammenhänge zwischen den Eigenschaften einer Wohnung und den Bewohnerkategorien zu modellieren. Wir verwenden die Evidenztheorie, um Wahrscheinlichkeiten vorherzusagen, auch wenn einige Informationen unvollständig sind.

---

## Vorbereitung
Zuerst werden die benötigten Bibliotheken importiert.

In [6]:
import pandas as pd
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter
from combined_mass.combined_mass import calculate_combined_mass

## Reinigung des Datensatzes
- **Bewohnerkategorie**: Der Datensatz enthält mehrere Spalten, mit verschiedenen Bewohnergruppen, wie DINK (Double Income No Kids) aber auch Alleinerziehend und Studierende, die die Zugehörigkeit zu einer Kategorie angeben. Diese werden in einer neuen Spalte `Bewohnerkategorie` zusammengefasst.
- **Datenbereinigung**: Um den Datensatz korrekt nutzen zu können, müssen wir den Fließtext durch Zahlen ersetzen, damit die KI mit diesen Daten arbeiten kann. Dafür werden "nein" und "ja" in numerische Werte (0 und 1) umgewandelt. Werte wo es mehr als ein Wert gibt wie bei der Miete, wurde der Mittelwert (gerundet) der beiden Werte genommen. Diese Funktionen sind in `data_cleaning/text_to_numeric.py` zu finden.

In [7]:
file_path = '../../data/Wohnungen_1.csv'
data = pd.read_csv(file_path, sep=';') # Gegebenen Datensatz einlesen

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

# Löschen von Daten, die nicht kategorisiert sind (dazu später mehr)
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
pd.set_option('future.no_silent_downcasting', True)
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1}).infer_objects(copy=False)

# Fließtext durch Ganzzahl ersetzen
data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0    726          276  ...          0          0           116   
13       1    726          226  ...          0          0           106   
23       0    776          276  ...          0          0           116   
27       0    226           50  ...          0          0            25   
40       0    226           50  ...          0          0            36   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

### Modelle erstellen und trainieren
Zuerst wird die Menge Omega mit den zu bestimmenden Attributen, d.h. der Bewohnerkategorie, bestimmt.

In [8]:
omega = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']

Jetzt können wir ein Beispiel mit Attributen angeben. Wie im Beispiel unten, wo Attribute für Schule wie folgt angegeben werden können: `'Schule': get_schule("nah")`. Die kombinierte Evidenz wird dann mit der Funktion `calculate_combined_mass()` berechnet, die diese aus unseren bereinigten Daten, den angegebenen Attributen und dem Omega berechnet. Diese selbstgeschriebene Funktion befindet sich in der Python-Datei `combined_mass/combined_mass.py`. Am Ende gibt die Methode `max_bel()` das Attribut des Omegas mit der höchsten Glaubwürdigkeit von allen zurück.

In [12]:
# Gegebene Attributkombinationen
attributes_values = {
    'Kindergarten': get_kindergarten("nah"),
    'Schule': get_schule("nah"),
}

# Berechne die kombinierte Evidenz
combined_mass = calculate_combined_mass(data, attributes_values, omega)

# Ausgabe der kombinierten Evidenz
print("Kombinierte Evidenz:", combined_mass)

# Wahrscheinlichste Hypothese basierend auf der kombinierten Evidenz
most_likely = combined_mass.max_bel()

# Extrahiere den Namen der Bewohnerklasse aus dem frozenset
if isinstance(most_likely, frozenset):
    most_likely = next(iter(most_likely), None)  # Extrahiere das einzige Element oder None, falls leer

print("\nAm wahrscheinlichsten ist:", most_likely)

Basismaße für Kindergarten=2: {set():0.6417910447761195; {'Kleinfamilie'}:0.23880597014925373; {'Studierende'}:0.05970149253731343; {'DINK'}:0.04477611940298507; {'Expatriate'}:0.014925373134328358; {'Alleinerziehende'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Schule=2: {{'Kleinfamilie'}:0.5223880597014925; set():0.35820895522388063; {'Studierende'}:0.1044776119402985; {'DINK'}:0.014925373134328358; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:0.9475465313028765; {'Studierende'}:0.047377326565143825; {'DINK'}:0.005076142131979696; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:0.9475465313028765; {'Studierende'}:0.047377326565143825; {'DINK'}:0.005076142131979696; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}

Am wahrscheinlichsten ist: Kleinfamilie


Es kann jedoch vorkommen, dass die Evidenz 0 ist, wenn die Basismaße nicht übereinstimmen. In diesem Fall gibt die Funktion einen Hinweis aus, dass keine Bewohnerkategorie vorhergesagt werden kann. Dies wird im folgenden Beispiel veranschaulicht.

In [14]:
attributes_values = {
    'Schule': get_schule("fern"),
    'Heizung': get_heizung("Gas"),
}

combined_mass = calculate_combined_mass(data, attributes_values, omega)
print("Kombinierte Evidenz:", combined_mass)
most_likely = combined_mass.max_bel()

if isinstance(most_likely, frozenset):
    most_likely = next(iter(most_likely), None)

print("\nAm wahrscheinlichsten ist:", most_likely)

Basismaße für Schule=0: {set():0.8507462686567164; {'Studierende'}:0.08955223880597014; {'DINK'}:0.05970149253731343; {'Kleinfamilie'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Heizung=-1: {set():1.0; {'Studierende'}:0.0; {'Kleinfamilie'}:0.0; {'DINK'}:0.0; {'Alleinerziehende'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Konflikt erkannt: Keine gemeinsame Hypothese zwischen den Massen.
Kombinierte Evidenz: {set():1.0}

Am wahrscheinlichsten ist: None
